In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from ml_utils import edaDF 

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [2]:
#Please change to your name.
name = "Vaibhavi"

#Please use this to control EDA. 
show_eda = False
if show_eda:
    df_eda = edaDF(df, "target")
    df_eda.fullEDA()


In [3]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

target  var_1  var_2  var_3  var_4  var_5  var_6  var_7  var_8  var_9  \
63        1  0.015  0.077  0.912  0.282  0.730  0.148  0.299  0.564  0.802   
87        1  0.242  0.078  0.858  0.596  0.332  0.712  0.493  0.323  0.504   
183       0  0.725  0.365  0.793  0.399  0.094  0.909  0.449  0.815  0.949   
135       1  0.169  0.665  0.996  0.488  0.742  0.223  0.735  0.278  0.600   
11        0  0.716  0.534  0.857  0.493  0.963  0.852  0.975  0.176  0.048   

     ...  var_191  var_192  var_193  var_194  var_195  var_196  var_197  \
63   ...    0.985    0.431    0.930    0.359    0.982    0.319    0.882   
87   ...    0.231    0.658    0.354    0.771    0.695    0.333    0.384   
183  ...    0.333    0.210    0.121    0.203    0.196    0.436    0.244   
135  ...    0.358    0.206    0.923    0.431    0.615    0.875    0.843   
11   ...    0.348    0.794    0.487    0.749    0.630    0.919    0.448   

     var_198  var_199  var_200  
63     0.906    0.382    0.251  
87     0.772    0.670    0.726  
183    0.676    0.505    0.581  
135    0.136    0.351    0.363  
11     0.112    0.050    0.385  

[5 rows x 201 columns]

### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [4]:
df.shape

(250, 201)

In [5]:
X = df.drop('target', axis=1)
y = df['target']

#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])


In [7]:
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best = grid_search.best_estimator_

### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [8]:
print(best.score(X_test, y_test))
print(best)

0.6
Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(min_samples_leaf=2, min_samples_split=5,
                                        random_state=42))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [9]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


c:\Users\vaibh\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


0.6388498562197986
0.6386329113924051
Vaibhavi 0.6387413838061018


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
- Applied GridSearchCV to identify the most effective hyperparameters for RandomForestClassifier, ensuring a balance between generalization and model complexity
- Used StandardScaler to standardize feature scales, which helps in improving the stability and performance of the model
- Opted for RandomForestClassifier due to its ensemble learning approach, which inherently helps in reducing overfitting
- Employed cross-validation during the process of tuning hyperparameters to robustly assess the model's performance